In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['HF_HOME'] = '/workspace/GCT634_final/huggingface'

In [2]:
# FastLanguageModel.from_pretrained?

In [3]:
from unsloth import FastLanguageModel
import torch

# fourbit_models = [
#     "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
#     "unsloth/Qwen3-4B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-8B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-14B-unsloth-bnb-4bit",
#     "unsloth/Qwen3-32B-unsloth-bnb-4bit",

#     # 4bit dynamic quants for superior accuracy and low memory use
#     "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
#     "unsloth/Phi-4",
#     "unsloth/Llama-3.1-8B",
#     "unsloth/Llama-3.2-3B",
#     "unsloth/Qwen3-4B",
#     "unsloth/Qwen3-8B"
#     "unsloth/Qwen3-14B"
#     "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B",
    max_seq_length = 768,   # Context length - can be longer, but uses more memory
    load_in_4bit = False,     # 4bit uses much less memory
    load_in_8bit = True,    # A bit more accurate, uses 2x memory
    full_finetuning = True, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)
# FastLanguageModel.from_pretrained?

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: You selected full finetuning support, but 4bit / 8bit is enabled - disabling LoRA / QLoRA.
==((====))==  Unsloth 2025.6.1: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


model.safetensors.index.json:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,  # Best to choose alpha = rank or rank*2
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,   # We support rank stabilized LoRA
#     loftq_config = None,  # And LoftQ
# )

In [ ]:
import pandas as pd
train_caption = pd.read_csv("data/DX7_YAMAHA_train_captions.csv", index_col=0)
train_caption_add = pd.read_csv("data/DX7_AllTheWeb_train_captions.csv", index_col=0)
test_caption = pd.read_csv("data/DX7_YAMAHA_test_captions.csv", index_col=0)
train_data = pd.read_csv("data/DX7_YAMAHA_train.csv", index_col=0)
train_data_add = pd.read_csv("data/DX7_AllTheWeb_train.csv", index_col=0)
test_data = pd.read_csv("data/DX7_YAMAHA_test.csv", index_col=0)
train_df = pd.merge(train_data, train_caption[['id', 'caption']], on='id', how='left')
train_df_add = pd.merge(train_data_add, train_caption_add[['id', 'caption']], on='id', how='left')
train_df = pd.concat([train_df, train_df_add])
test_df = pd.merge(test_data, test_caption[['id', 'caption']], on='id', how='left')
train_df

In [ ]:
train_df = train_df[(train_df['inaudible'] == False) & ~train_df['name'].str.contains('NULL') & ~train_df['has_fixed_freqs']]
test_df = test_df[(test_df['inaudible'] == False) & ~test_df['name'].str.contains('NULL') & ~test_df['has_fixed_freqs']]
print(len(train_df), len(test_df))
KEYS_TO_REMOVE = ['name', 'has_fixed_freqs', 'fixed_freq']

In [ ]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_df[['id', 'caption', 'patch_data']])
test_dataset = Dataset.from_pandas(test_df[['id', 'caption', 'patch_data']])

In [ ]:
from prompt import zeroshot_prompt
import ast
from utils import serialize_specs
def proprocess(examples, prompt = zeroshot_prompt):
    conversations = []
    for caption, patch_data in zip(examples["caption"], examples["patch_data"]):
        question = prompt.format(prompt=caption)
        specs = ast.literal_eval(patch_data)
        for key in KEYS_TO_REMOVE:
            specs.pop(key, None)
        solution = '```python\n' + 'specs = ' +serialize_specs(specs) + '```\n'
        conversations.append([
            {"role" : "user",      "content" : question},
            {"role" : "assistant", "content" : solution},
        ])
    return { "conversations": conversations, }

In [ ]:
train_dataset = tokenizer.apply_chat_template(
    train_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
test_dataset = tokenizer.apply_chat_template(
    test_dataset.map(proprocess, batched = True)["conversations"],
    tokenize = False,
)
train_dataset = pd.Series(train_dataset)
test_dataset = pd.Series(test_dataset)
train_dataset.name = "text"
test_dataset.name = "text"
train_dataset = Dataset.from_pandas(pd.DataFrame(train_dataset)).shuffle()
test_dataset = Dataset.from_pandas(pd.DataFrame(test_dataset))

In [ ]:
# lengths = [len(tokenizer(train_dataset[i]['text'])['input_ids']) for i in range(len(train_dataset))]
# print(max(lengths))

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset, # Can set up evaluation!
    args = SFTConfig(
        eval_strategy="steps",
        eval_steps = 100,
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 100,
        # warmup_ratio = 0.01,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 1e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        logging_dir = "logs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        # bf16 = True,
    ),
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#0.180400	0.266946
#0.203500	0.259421
#  [2270/2288 26:59 < 00:12, 1.40 it/s, Epoch 0.99/1]
# Step	Training Loss	Validation Loss
# 100	0.312600	0.444091
# 200	0.249000	0.314182
# 300	0.286600	0.288614
# 400	0.321900	0.279245
# 500	0.187000	0.260607
# 600	0.301500	0.252575
# 700	0.232400	0.246626
# 800	0.250500	0.239504
# 900	0.209000	0.228961
# 1000	0.262700	0.221790
# 1100	0.295300	0.215801
# 1200	0.236800	0.205192
# 1300	0.220500	0.202858
# 1400	0.211400	0.195314
# 1500	0.199600	0.189298
# 1600	0.146000	0.181624
# 1700	0.230300	0.176836
# 1800	0.200400	0.171057
# 1900	0.133000	0.168132
# 2000	0.173400	0.165630
# 2100	0.181000	0.163183
# 2200	0.155600	0.162581

In [ ]:
caption = test_df.iloc[23]['caption']
messages = [
    {"role" : "user", "content" : zeroshot_prompt.format(prompt=caption)}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = False, # Disable thinking
)
print(caption)

from transformers import TextStreamer
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # temperature = 1.0, #, top_p = 0.8, top_k = 20, # For non thinking
    # do_sample = False,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
from pydx7 import load